In [33]:
import sql_tools
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick_ohlc

fire = sql_tools.IcePick(sqlite3, 'fire.db')

In [34]:
def chart_candles(candles):
        ohlc = []
        for index in candles.index:
            time = index
            open_= candles.loc[index, 'open']
            high = candles.loc[index, 'high']
            low= candles.loc[index, 'low']
            close = candles.loc[index, 'close']
            ohlc.append([time, open_, high, low, close])
        
        fig = plt.figure()
        fig.set_size_inches(13, 7)
        ax1 = plt.subplot2grid((1,1), (0,0))
        candlestick_ohlc(ax1, ohlc, width=0.4)
        plt.show()

In [35]:
pair_names = list(fire.read_sql("SELEcT * from sqlite_master where type = 'table'")['name'])

In [36]:
pair_data = {}
def get_data(currency):
    data = fire.read_sql("SELECT * FROM [{}]".format(currency))
    data.index = data['time']
    data.sort_values('time', inplace = True)
    return data

for pair_name in pair_names:
    pair_data[pair_name] = get_data(pair_name)

In [37]:
print(list(pair_data.keys()))

['BTC-EUR', 'LTC-EUR', 'LTC-BTC', 'BTC-GBP', 'BTC-USD', 'ETH-USD', 'LTC-USD', 'ETH-BTC', 'ETH-EUR']


In [38]:
def win_condition(candles_in, prediction_gap = 2):
    result = []
    for i, index in enumerate(candles.index):
        if i < prediction_gap:
            result.append(0)
        if i >= prediction_gap :
            current_low = candles.loc[index, 'low']
            last_high = candles.iloc[i - prediction_gap]['low']
            if current_low < last_high:
                result.append(1)
            if current_low >= last_high:
                result.append(0)
    return result




In [47]:
def get_diffs(x):
    last_index = x.index[0]
    for index in x.index:
        x.loc[index, 'time_diff'] = index - last_index

    return x

In [51]:
for pair_name in pair_data.keys():
    pair_data[pair_name] = get_diffs(pair_data[pair_name])

In [55]:
pair_data.keys()

dict_keys(['BTC-EUR', 'LTC-EUR', 'LTC-BTC', 'BTC-GBP', 'BTC-USD', 'ETH-USD', 'LTC-USD', 'ETH-BTC', 'ETH-EUR'])

In [62]:
x = pair_data['LTC-USD']
x[x['time_diffs'] > 60]

,time,low,high,open,close,volume,time_string,time_diffs,time_diff
time,,,,,,,,,
1513451340,1513451340,298.9,298.95,298.9,298.95,9.631034,2017-12-16 11:09:00,43080,55080.0
